In [1]:
import torch
import torchvision
import math
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm_notebook
from torch.utils.tensorboard import SummaryWriter
from torch import nn


from GWD_data import WheatDataset
from metric import calculate_image_precision
from utils import collate_fn, plot_boxes, get_model_name, format_prediction_string, remove_blanks, is_contain_blanks
from config import config


In [2]:
conf = config()
torch.random.manual_seed(5)
writer = SummaryWriter("runs/May11_23-57-42_DESKTOP-ELPLUSQ") #


WD_Train = WheatDataset(conf)
WD_Valid = WheatDataset(conf, train=False)
WD_Train_Loader = torch.utils.data.DataLoader(WD_Train, batch_size=conf.BATH_SIZE, shuffle=True, collate_fn=collate_fn)
WD_Valid_Loader = torch.utils.data.DataLoader(WD_Valid, batch_size=conf.BATH_SIZE, shuffle=True, collate_fn=collate_fn)

In [3]:
class ModelWithLoss(nn.Module):
    def __init__(self, model, debug=False):
        super().__init__()
        self.criterion = FocalLoss()
        self.model = model
        self.debug = debug

    def forward(self, imgs, annotations, obj_list=None):
        _, regression, classification, anchors = self.model(imgs)
        if self.debug:
            cls_loss, reg_loss = self.criterion(classification, regression, anchors, annotations,
                                                imgs=imgs, obj_list=obj_list)
        else:
            cls_loss, reg_loss = self.criterion(classification, regression, anchors, annotations)
        return cls_loss, reg_loss

In [4]:
import torch.nn.functional as F
from EfficientDet.backbone import EfficientDetBackbone
from EfficientDet.efficientdet.loss import FocalLoss

EPOCH = 0 #int(model_path.split('/')[-1].split('_')[2])+1 if model_path else 0

#num_classes=1
compound_coef=0 #default level
#num_classes=num_classes

GWD_effdet = EfficientDetBackbone( compound_coef=compound_coef)
state_dict = torch.load('EfficientDet/efficientdet-d2.pth')


try:
    ret = GWD_effdet.load_state_dict(state_dict)  #Strict=False ignores non matching keys
except RuntimeError as e:
    print(f'[Warning] Ignoring {e}')
    print('[Warning] Don\'t panic if you see this, this might be because you load a pretrained weights with different number of classes. The rest of the weights should be loaded already.')


debug=False
GWD_effdet = ModelWithLoss(GWD_effdet, debug=debug)


GWD_effdet_model.to(conf.DEVICE)
params = [p for p in GWD_effdet_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001, weight_decay=0.0001)      #These very the default values in source code

[Warning] Ignoring Error(s) in loading state_dict for EfficientDetBackbone:
	Missing key(s) in state_dict: "backbone_net.model._blocks.1._expand_conv.conv.weight", "backbone_net.model._blocks.1._bn0.weight", "backbone_net.model._blocks.1._bn0.bias", "backbone_net.model._blocks.1._bn0.running_mean", "backbone_net.model._blocks.1._bn0.running_var". 
	Unexpected key(s) in state_dict: "bifpn.3.p6_w1", "bifpn.3.p5_w1", "bifpn.3.p4_w1", "bifpn.3.p3_w1", "bifpn.3.p4_w2", "bifpn.3.p5_w2", "bifpn.3.p6_w2", "bifpn.3.p7_w2", "bifpn.3.conv6_up.depthwise_conv.conv.weight", "bifpn.3.conv6_up.pointwise_conv.conv.weight", "bifpn.3.conv6_up.pointwise_conv.conv.bias", "bifpn.3.conv6_up.bn.weight", "bifpn.3.conv6_up.bn.bias", "bifpn.3.conv6_up.bn.running_mean", "bifpn.3.conv6_up.bn.running_var", "bifpn.3.conv6_up.bn.num_batches_tracked", "bifpn.3.conv5_up.depthwise_conv.conv.weight", "bifpn.3.conv5_up.pointwise_conv.conv.weight", "bifpn.3.conv5_up.pointwise_conv.conv.bias", "bifpn.3.conv5_up.bn.weight", 

NameError: name 'GWD_effdet_model' is not defined

In [ ]:
_iter=len(WD_Train_Loader)*(EPOCH)
clip_max_norm=0.1  #Default value in their code

for i in tqdm_notebook(range(EPOCH, 50, 1)):
    
    _epoch_loss=0
    _ = GWD_detr_Model.train()
    _ = criterion.train()
    
    for images, targets in tqdm_notebook(WD_Train_Loader):

        images = [torch.tensor(image, dtype = torch.float32).to(conf.DEVICE) for image in images]
        targets = [{k: torch.tensor(v).to(conf.DEVICE) for k, v in target.items()} for target in targets]
        
        #Main fwd pass and loss calc  
        outputs = GWD_detr_Model(images)
        loss_dict = criterion(outputs, targets)
        weight_dict = criterion.weight_dict
        summed_loss_value = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)


        if math.isfinite(summed_loss_value):
            optimizer.zero_grad()
            summed_loss_value.backward()
            if clip_max_norm > 0:
                torch.nn.utils.clip_grad_norm_(GWD_detr_Model.parameters(), clip_max_norm)
            optimizer.step()
            _epoch_loss+=summed_loss_value
            
            if(_iter%1 == 0):
                print("".join([k+":"+str(v.data.cpu().numpy())+", " for k,v in loss_dict.items()]))

        else:
            print('Loss is undefined:',summed_loss_value,'   skipping BackProp for step no:',_iter)
            print(loss_dict)


        writer.add_scalar('Running Loss/Summed', summed_loss_value, _iter)
        writer.add_scalar('Running Loss/CE', loss_dict['loss_ce'].item(), _iter)
        writer.add_scalar('Running Loss/Classifier', loss_dict['class_error'].item(), _iter)
        writer.add_scalar('Running Loss/Box_Regress', loss_dict['loss_bbox'].item(), _iter)
        writer.add_scalar('Running Loss/loss_giou', loss_dict['loss_giou'].item(), _iter)
        writer.add_scalar('Running Loss/cardinality_error', loss_dict['cardinality_error'].item(), _iter)
         
        _iter+=1
        
    print('Saving model at epoch '+str(i)+', step '+str(_iter))
    torch.save(GWD_detr_Model, 'model-detr/'+get_model_name(i, 0, _epoch_loss))
    torch.cuda.empty_cache()
    gc.collect()
    break